In [1]:
!pip install langchain langchain-community
!pip install sentence-transformers
!pip install chromadb
!pip install transformers
!pip install torch torchvision torchaudio
!pip install huggingface_hub
!pip install pypdf
!pip install bitsandbytes
!pip install llamaparse

ERROR: Could not find a version that satisfies the requirement llamaparse (from versions: none)
ERROR: No matching distribution found for llamaparse


In [2]:
!pip install llama-parse

In [3]:
import os
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HF_TOKEN")

# Login to Hugging Face
login(token=secret_value_0)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
!pip show transformers
!pip show sentence-transformers
!pip show tf-keras


Name: transformers
Version: 4.45.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /opt/conda/lib/python3.10/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: kaggle-environments, sentence-transformers
Name: sentence-transformers
Version: 3.2.0
Summary: State-of-the-Art Text Embeddings
Home-page: https://www.SBERT.net
Author: 
Author-email: Nils Reimers <info@nils-reimers.de>, Tom Aarsen <tom.aarsen@huggingface.co>
License: Apache 2.0
Location: /opt/conda/lib/python3.10/site-packages
Requires: huggingface-hub, Pillow, scikit-learn, scipy, torch, tqdm, transformers
Required-by: 
Name: tf_keras

In [5]:
!pip install --upgrade transformers sentence-transformers tf-keras


In [6]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import chromadb
from transformers import LlamaForCausalLM, LlamaTokenizer
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoConfig, pipeline
from huggingface_hub import login
import os
from llama_parse import LlamaParse
from transformers import AutoTokenizer, AutoModel


2024-10-14 17:02:52.237610: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-14 17:02:52.258318: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-14 17:02:52.264765: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
from sentence_transformers import SentenceTransformer

In [10]:
import nest_asyncio
secret_value_1 = user_secrets.get_secret("llamaParse")
nest_asyncio.apply()
input_dir = "/kaggle/input/"
parser = LlamaParse(api_key=secret_value_1)
# List to store the loaded data
pdf_data = []
# Walk through the input directory and its subdirectories
for root, dirs, files in os.walk(input_dir):
    for file in files:
        if file.endswith(".pdf"):
            file_path = os.path.join(root, file)
            data_load = PyPDFLoader(file_path)
            data = data_load.load()
            pdf_data.append(data)

In [11]:
print(pdf_data[0][0])

page_content='cancer.org | 1.800.227.2345
 
Understanding a Breast Cancer
Diagnosis  
If you’ve been diagnosed with breast cancer, you’ve probably heard a lot of different
terms used to describe your cancer. Doctors use information from your breast biopsy to
learn a lot of important things about the exact kind of cancer you have. You may also
need more tests to get more details, such as the stage of the cancer or how fast it’s
growing.
Breast Cancer Grade and Other Tests
Doctors use information from your breast biopsy to learn a lot of important things about
the exact kind of breast cancer you have.
Breast Cancer Grades●
Breast Cancer Ploidy and Cell Proliferation●
Breast Cancer Hormone Receptor Status●
Breast Cancer HER2 Status●
Breast Cancer Gene Expression Tests●
Other Breast Cancer Gene, Protein, and Blood Tests●
Understanding Your Pathology Report●
Stages and Outlook (Prognosis)
If you have been diagnosed with breast cancer, tests will be done to find out the extent
(stage) of the

In [14]:
page_contents = [doc for i in range(len(pdf_data)) for doc in pdf_data[i]]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
documents = text_splitter.split_documents(page_contents)
len(documents)

6269

In [15]:
from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('all-mpnet-base-v2')
pdf_embedding = embedder.encode([doc.page_content for doc in documents])


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/196 [00:00<?, ?it/s]

In [16]:
client = chromadb.Client()

# Create a collection for PDF document embeddings
pdf_collection = client.create_collection("pdf_document")

# Generate unique IDs for each document
ids = [str(i) for i in range(len(documents))]

# Add the PDF text and its embedding to Chroma collection
# Use the content of each Document object and include the generated IDs
pdf_collection.add(
    ids=ids,  # Unique IDs for each document
    documents=[doc.page_content for doc in documents],  # Store the raw text
    embeddings=pdf_embedding  # Store the corresponding embedding
)

In [17]:
def query_chroma(query):
    # Generate embedding for the user query
    query_embedding = embedder.encode([query])

    # Retrieve relevant documents from Chroma
    results = pdf_collection.query(
        query_embeddings=query_embedding,
        n_results=10  # Get top 5 relevant documents
    )
    retrieved_docs = results['documents']
    return retrieved_docs

In [18]:
model_id = 'meta-llama/Llama-3.1-8B-Instruct' 
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [19]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    low_cpu_mem_usage = True
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [20]:
model_directory = 'saved_model'

# Save the model and tokenizer
model.save_pretrained(model_directory)
tokenizer.save_pretrained(model_directory)

('saved_model/tokenizer_config.json',
 'saved_model/special_tokens_map.json',
 'saved_model/tokenizer.json')

In [21]:
query = "The Types of non-cancerous breast conditions"
retrieved_docs = query_chroma(query)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [22]:
retrieved_docs

[['cancer.org | 1.800.227.2345\n \nNon-cancerous Breast Conditions\xa0\nBenign (non-cancerous) breast conditions are very common, and most women have\nthem. In fact, most breast changes are benign. Unlike\xa0breast cancers, benign breast\nconditions are not life-threatening. But some are linked with a higher risk of getting\nbreast cancer later on.\nSome benign breast changes may cause signs or symptoms (such as breast lumps,',
  'cancer screenings as recommended, as these conditions do not eliminate \nthe risk of breast cancer.  \n \n \n \nQ1091. What are the common types of benign breast conditions?    \nA. Common types include fibrocystic changes, fibroadenomas, sclerosing \nadenosis, intraductal papillomas, and mastitis, among others.  \n \n Q1092. How do fibrocystic changes differ from fibroadenomas?',
  '304  A. Support groups provide emotional support, shared experiences, and \nvaluable information, helping women cope with the psychological aspects of their diagnosis and treatme

In [23]:
def format_chat_template(question, retrieved_context):
    # Template to answer the question with strict relevance
    chat_template = f"""
Context:
{retrieved_context}

Instructions:
"You are a helpful assistant that is an expert at extracting the most useful information from the given context above. Also bring in extra relevant infromation to the user query from outside the given context if there any."
"if there any refrences or links in the context just ignore it"
Question:
{question}

Answer:

"""
    return chat_template

In [24]:
retrieved_context = " "
for i in range(len(retrieved_docs)):
    retrieved_context =retrieved_context.join(retrieved_docs[i])

In [25]:
# Join retrieved documents into a single context
#retrieved_context = " ".join(retrieved_docs)

# Format the input using the chat template
formatted_input = format_chat_template(query, retrieved_context)

formatted_input

'\nContext:\ncancer.org | 1.800.227.2345\n \nNon-cancerous Breast Conditions\xa0\nBenign (non-cancerous) breast conditions are very common, and most women have\nthem. In fact, most breast changes are benign. Unlike\xa0breast cancers, benign breast\nconditions are not life-threatening. But some are linked with a higher risk of getting\nbreast cancer later on.\nSome benign breast changes may cause signs or symptoms (such as breast lumps, cancer screenings as recommended, as these conditions do not eliminate \nthe risk of breast cancer.  \n \n \n \nQ1091. What are the common types of benign breast conditions?    \nA. Common types include fibrocystic changes, fibroadenomas, sclerosing \nadenosis, intraductal papillomas, and mastitis, among others.  \n \n Q1092. How do fibrocystic changes differ from fibroadenomas? 304  A. Support groups provide emotional support, shared experiences, and \nvaluable information, helping women cope with the psychological aspects of their diagnosis and treatme

In [26]:
inputs = tokenizer(formatted_input, return_tensors="pt").to(model.device)

In [27]:
import re

output = model.generate(
    **inputs,
    max_new_tokens=150,
    no_repeat_ngram_size=4,
    repetition_penalty=1.2,
    top_p=0.95,
    temperature=0.7,
    eos_token_id=tokenizer.eos_token_id,
)

response = tokenizer.decode(output[0], skip_special_tokens=True)

answer_start = response.find("Answer:") + len("Answer: ")

only_answer = response[answer_start:].strip()

cleaned_answer = re.sub(r'\s+', ' ', only_answer)  # Replace multiple spaces with a single space
cleaned_answer = re.sub(r'[^\w\s.,;\[\]\(\)]', '', cleaned_answer)
sentences = re.findall(r'.*?[.!?]', cleaned_answer)
if sentences:
    cleaned_answer = ' '.join(sentences)

print(cleaned_answer)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


There are several types of noncuterous breast conditions which includes but not limited to  fibrocystic changes also known as fibrosism andor simple cysstis(non proliferative lesion)  hyperplasia, including both ductal and lobular types   Phyllodes Tumor  Adenosis  Intradermal Papilomas (note this type have been included under other catagories by different author) Note the following points about these conditions  While generally considered benign, some of these conditions can still lead to increased risk of developing breat cancer over time.


In [28]:
response

'\nContext:\ncancer.org | 1.800.227.2345\n \nNon-cancerous Breast Conditions\xa0\nBenign (non-cancerous) breast conditions are very common, and most women have\nthem. In fact, most breast changes are benign. Unlike\xa0breast cancers, benign breast\nconditions are not life-threatening. But some are linked with a higher risk of getting\nbreast cancer later on.\nSome benign breast changes may cause signs or symptoms (such as breast lumps, cancer screenings as recommended, as these conditions do not eliminate \nthe risk of breast cancer.  \n \n \n \nQ1091. What are the common types of benign breast conditions?    \nA. Common types include fibrocystic changes, fibroadenomas, sclerosing \nadenosis, intraductal papillomas, and mastitis, among others.  \n \n Q1092. How do fibrocystic changes differ from fibroadenomas? 304  A. Support groups provide emotional support, shared experiences, and \nvaluable information, helping women cope with the psychological aspects of their diagnosis and treatme